In [2]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [15]:
import praw
import pandas as pd
import time

In [25]:

CLIENT_ID = "_WoTwn9oo6HKHJeRIdjdNQ"  # <-- My Client ID
CLIENT_SECRET = "Mxdg0bbYtxu75puXpw5JyQfDdQ7yRQ" # <--- My SECRET!
USERNAME = "Double_Analyst_7896"
PASSWORD = "Rashkavmar1" # <--- My PASSWORD!
USER_AGENT = "MyKoreaFoodBot/0.2 by u/Double_Analyst_7896"



In [26]:
SUBREDDIT_NAME = "KoreanFood"
POST_LIMIT = 500 

def get_reddit_top_posts(client_id, client_secret, username, password, user_agent, subreddit, limit):
    """Fetches top posts and comments from a subreddit."""
    print(f"Connecting to Reddit as {username}...")
    try:
        reddit = praw.Reddit(
            client_id=client_id,
            client_secret=client_secret,
            username=username,
            password=password,
            user_agent=user_agent,
        )
        if reddit.read_only:
             print("  !! Authentication failed! Connection is read-only. Check credentials. !!")
             return []
        else:
             print(f"  Successfully connected as: {reddit.user.me()}")
    except Exception as e:
        print(f"  Failed to connect: {e}")
        return []

    print(f"Fetching 'top' posts from r/{subreddit} (limit: {limit})...")
    subreddit_instance = reddit.subreddit(subreddit)
    posts_data = []

    try:
        
        top_posts = subreddit_instance.top(time_filter='year', limit=limit)

        for post in top_posts:
            post_info = {
                "id": post.id,
                "title": post.title,
                "score": post.score,
                "url": post.url,
                "created_utc": post.created_utc,
                "selftext": post.selftext,
                "num_comments": post.num_comments,
                "flair": post.link_flair_text
            }
            post.comments.replace_more(limit=0)
            top_comments = [comment.body for comment in post.comments.list()[:5]] 
            post_info["comments"] = " | ".join(top_comments)
            posts_data.append(post_info)
            print(f"  Fetched post: {post.id} ({len(posts_data)}/{limit})")
            time.sleep(0.5) 

    except Exception as e:
        print(f"An error occurred during fetch: {e}")

    return posts_data


reddit_koreanfood_posts = get_reddit_top_posts(
    CLIENT_ID, CLIENT_SECRET, USERNAME, PASSWORD, USER_AGENT,
    SUBREDDIT_NAME, POST_LIMIT
)

if reddit_koreanfood_posts:
    reddit_kf_df = pd.DataFrame(reddit_koreanfood_posts)
    print(f"\nCollected {len(reddit_kf_df)} posts from r/KoreanFood.")
    print(reddit_kf_df.head())

    reddit_kf_df['full_text'] = reddit_kf_df['title'] + " " + reddit_kf_df['selftext'] + " " + reddit_kf_df['comments']


    reddit_kf_df.to_csv("korean_food_subreddit_raw.csv", index=False)
    print("Saved raw Reddit data to 'korean_food_subreddit_raw.csv'")
else:
    print("No posts were collected. Check your credentials or subreddit name.")

Connecting to Reddit as Double_Analyst_7896...
  Successfully connected as: Double_Analyst_7896
Fetching 'top' posts from r/KoreanFood (limit: 500)...
  Fetched post: 1g9rszk (1/500)
  Fetched post: 1dd3zl9 (2/500)
  Fetched post: 1hde333 (3/500)
  Fetched post: 1f220ih (4/500)
  Fetched post: 1hpreyp (5/500)
  Fetched post: 1f5mozj (6/500)
  Fetched post: 1dkwats (7/500)
  Fetched post: 1h5skhe (8/500)
  Fetched post: 1fmy3y6 (9/500)
  Fetched post: 1cpepcp (10/500)
  Fetched post: 1gw2h5z (11/500)
  Fetched post: 1d4qr77 (12/500)
  Fetched post: 1ezt5dw (13/500)
  Fetched post: 1ethqpv (14/500)
  Fetched post: 1gxxl4n (15/500)
  Fetched post: 1ggpg96 (16/500)
  Fetched post: 1g0udtd (17/500)
  Fetched post: 1grlhby (18/500)
  Fetched post: 1fle9i6 (19/500)
  Fetched post: 1f7ojz7 (20/500)
  Fetched post: 1hn1uhs (21/500)
  Fetched post: 1fptuuv (22/500)
  Fetched post: 1h2nnk5 (23/500)
  Fetched post: 1dv3zdb (24/500)
  Fetched post: 1gcxfp8 (25/500)
  Fetched post: 1har3ic (26/500)


In [28]:
reddit_kf_df.tail()

,id,title,score,url,created_utc,selftext,num_comments,flair,comments,full_text
495,1fmppfv,Difference in Gochujang,98,https://i.redd.it/7x26qqpy2cqd1.jpeg,1.726999e+09,Just got back from a two week long Korean vaca...,14,questions,The top one has rice from Korea in it (우리쌀) | ...,Difference in Gochujang Just got back from a t...
496,1fbqpev,Dolsot Bibimbap,95,https://i.redd.it/cbtc74lhwind1.jpeg,1.725774e+09,,4,Traditional,This my wife’s favorite dish! She white but ma...,Dolsot Bibimbap This my wife’s favorite dis...
497,1dtvvlp,What is this side dish at the KBBQ?,94,https://i.redd.it/0apymcvs06ad1.jpeg,1.719952e+09,Not sure what this banchan is. It didn't taste...,19,questions,Mung bean starch jelly. \n\nAcorn is brown but...,What is this side dish at the KBBQ? Not sure w...
498,1d5ieit,What can I do to utilize and / or preserve thi...,94,https://www.reddit.com/gallery/1d5ieit,1.717231e+09,"Seriously, this is a lot and I don’t want it t...",34,questions,You kind of have several stages of preservatio...,What can I do to utilize and / or preserve thi...
499,1g2jqvd,Bossam time,94,https://i.redd.it/eq7m1hfozgud1.jpeg,1.728802e+09,,2,Meat foods 🥩🍖,"Wow, it looks so delicious. I want to drink Ma...","Bossam time Wow, it looks so delicious. I wan..."
